In [0]:
'''
Iniciando build_accidents_fact
'''

'\nIniciando build_accidents_fact\n'

In [0]:
# Definindo o fato
# Número de acidentes

# Definindo as dimensões
'''
Quem: Quais os operadores dos acidentes
Quando: Quais os dias/anos com mais acidente
Quando: Qual a fase de voo com mais acidente
Onde: Qual o lugar onde ocorrem os acidentes
O que: Quais as aeronaves mais acidentadas
Porque:
Como:

Utilizando uma abordagem ROLAP, cada fato e dimensão do esquema de um DW é implementado em uma tabela

Em uma tabela de fatos, cada medida é um atributo (ou coluna), e são adicionadas colunas (chaves estrangeiras) para referenciar cada uma das dimensões que descreve esse fato

Para projetar um DW seguindo um esquema estrela, para cada fato deve haver apenas uma tabela (sem redundância alguma), e essa tabela fato deve estar relacionada a exatamente uma tabela para cada dimensão que descreve aquele fato. Isso determina que as dimensões sejam necessariamente desnormalizadas, ou seja, as hierarquias de cada dimensão são representadas por atributos na tabela dimensão.
'''

'\nQuem: Quais os operadores dos acidentes\nQuando: Quais os dias/anos com mais acidente\nQuando: Qual a fase de voo com mais acidente\nOnde: Qual o lugar onde ocorrem os acidentes\nO que: Quais as aeronaves mais acidentadas\nPorque:\nComo:\n\nUtilizando uma abordagem ROLAP, cada fato e dimensão do esquema de um DW é implementado em uma tabela\n\nEm uma tabela de fatos, cada medida é um atributo (ou coluna), e são adicionadas colunas (chaves estrangeiras) para referenciar cada uma das dimensões que descreve esse fato\n\nPara projetar um DW seguindo um esquema estrela, para cada fato deve haver apenas uma tabela (sem redundância alguma), e essa tabela fato deve estar relacionada a exatamente uma tabela para cada dimensão que descreve aquele fato. Isso determina que as dimensões sejam necessariamente desnormalizadas, ou seja, as hierarquias de cada dimensão são representadas por atributos na tabela dimensão.\n'

Importando funções comuns para uso no notebook

In [0]:
%run
./shared_gold_functions

Inicializando sessão spark, definindo o schema da tabela final e importando dados da tabela silver

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql import functions as F
import urllib

# Inicializa a sessão Spark
spark = SparkSession.builder.appName("accidents_analysis").getOrCreate()

schema = StructType([
   StructField("index", IntegerType(), False),
   StructField("aircraft_id", StringType(), False),
   StructField("other_fatalities", IntegerType(), True),
   StructField("aircraft_damage", StringType(), True),
   StructField("category", StringType(), True),
   StructField("approx_location", StringType(), True),
   StructField("phase", StringType(), True),
   StructField("nature", StringType(), True),
   StructField("departure_airport_ICAO", StringType(), True),
   StructField("departure_airport_IATA", StringType(), True),
   StructField("destination_airport_ICAO", StringType(), True),
   StructField("destination_airport_IATA", StringType(), True),
   StructField("nb_fatalities", IntegerType(), True),
   StructField("nb_occupants", IntegerType(), True),
   StructField("country_code", StringType(), True),
   StructField("investigation_agency", StringType(), True),
   StructField("confidence_rating", StringType(), True),
   StructField("date_id", StringType(), True),
   StructField("time_id", StringType(), True),])

accidents_silver_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/silver_database.db/silver_accidents')

accidents_silver_df.display()

index,date,time,aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hrs,cycles,other_fatalities,aircraft_damage,category,approx_location,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating,aircraft_type_code,weekday,nb_fatalities,nb_occupants,country_or_region,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO
386881,2024-04-23,1345,Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,"Chapecó Airport, SC","Erechim-Comandante Kraemer Airport, RS",CENIPA,"Information is only available from news, social media or unofficial sources",LJ75,Tuesday,0,5,Brazil,XAP,SBCH,ERM,SSER
354203,2024-03-10,1452,IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,"Fort Lauderdale International Airport, FL","Hot Springs-Ingalls Field, VA",NTSB,Information verified through data from accident investigation authorities,ASTR,Sunday,5,5,USA,FLL,KFLL,HSP,KHSP
318650,2023-05-19,1100,Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",SC7,Friday,0,2,Uganda,null,null,null,null
318643,2023-07-08,1500,Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,0,Substantial,Accident,Vårgårda,Take off,Private,null,null,SHK,"Information is only available from news, social media or unofficial sources",AN2,Saturday,0,5,Sweden,null,null,null,null
318648,2023-06-04,1500,Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,null,"Islip-Long Island MacArthur Airport, NY",NTSB,Information verified through data from accident investigation authorities,C560,Sunday,4,4,USA,null,null,ISP,KISP
389190,2024-05-31,null,Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,N'Djamena Airport,Abuja-Nnamdi Azikiwe International Airport,null,Information verified through data from accident investigation authorities,A332,Friday,0,null,Chad,NDJ,FTTJ,ABV,DNAA
343377,2023-08-09,null,Lockheed CC-130H Hercules,Royal Canadian Air Force (RCAF),130337,382-4584,null,null,null,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",C130,Wednesday,0,null,Canada,null,null,null,null
351839,2024-02-18,2009,Boeing 767-332ER (WL),Delta Air Lines,N176DZ,29697/745,1999,null,null,null,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,"New York-John F. Kennedy International Airport, NY",Bogotá-Eldorado Airport,Aerocivil,"Information is only available from news, social media or unofficial sources",B763,Sunday,0,null,Atlantic Ocean,JFK,KJFK,BOG,SKBO
318674,2023-04-17,null,Antonov An-12BK,Al Quwwat al-Jawwiya As-Sudaniya (Sudanese Air Force),9977,9346302,null,null,null,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",AN12,Monday,0,0,Sudan,null,null,null,null
344872,2023-08-24,1542,Viking Air DHC-6 Twin Otter 400,Air Antilles,F-OMYS,971,null,null,null,null,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,Pointe-à-Pitre-Le Raizet Airport,Saint-Barthélémy-Rémy de Haenen Airport,BEA,"Information is only available from news, social media or unofficial sources",DHC6,Thursday,0,6,null,PTP,TFFR,SBH,TFFJ


## Processamento

Na etapa de processamento, vamos:
- Recuperar os dados de silver_accidents
- Para cada relacionamento com dimensão:
  - Atribuir as FK para as tabelas de dimensão definidas previamente
  - Selecionar as colunas de interesse
- Reordenar as colunas
- Salvar no novo schema


Começamos pelas Aeronaves. Vamos:
- Selecionar as colunas que definem uma aeronave num acidente (aircraft_type, owner_operator, registration e msn)
- Buscar o aircraft_id associado na tabela gold_aircraft_type_dim via join entre as tabelas accidents_silver e gold_aircraft_type_dim


In [0]:
# Buscando o id das aeronaves com base no aircraft_type, owner_operator, registration e msn
aircraft_gold_df = spark.read.format('delta').load('dbfs:/user/hive/warehouse/gold_database.db/gold_aircraft_type_dim')

#aircraft_gold_df.display()

# Criar coluna concatenando aircraft_type, owner_operator, registration e msn no df de aircraft
aircraft_gold_df = aircraft_gold_df.withColumn('full_aircraft_id', F.concat_ws("",'aircraft_type', 'owner_operator', 'registration', 'msn'))

# Criar coluna concatenando aircraft_type, owner_operator, registration e msn no df de accidents
accidents_silver_df = accidents_silver_df.withColumn('full_aircraft_id', F.concat_ws("",'aircraft_type', 'owner_operator', 'registration', 'msn'))

#accidents_silver_df.display()

cond = [aircraft_gold_df.full_aircraft_id == accidents_silver_df.full_aircraft_id]

accidents_gold_fact = accidents_silver_df.join(aircraft_gold_df,cond)

accidents_gold_fact.display()

index,date,time,aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hrs,cycles,other_fatalities,aircraft_damage,category,approx_location,phase,nature,departure_airport,destination_airport,investigating_agency,confidence_rating,aircraft_type_code,weekday,nb_fatalities,nb_occupants,country_or_region,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO,full_aircraft_id,aircraft_type,owner_operator,registration,msn,year_of_manufacture,engine_model,total_airframe_hours,cycles,manufacturer_country_name,year_of_first_flight_of_model,aircraft_id,full_aircraft_id
386881,2024-04-23,1345,Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,"Chapecó Airport, SC","Erechim-Comandante Kraemer Airport, RS",CENIPA,"Information is only available from news, social media or unofficial sources",LJ75,Tuesday,0,5,Brazil,XAP,SBCH,ERM,SSER,Learjet 75Cimed & Co SAPP-DYB45-565,Learjet 75,Cimed & Co SA,PP-DYB,45-565,null,null,null,null,USA,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66,Learjet 75Cimed & Co SAPP-DYB45-565
354203,2024-03-10,1452,IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,"Fort Lauderdale International Airport, FL","Hot Springs-Ingalls Field, VA",NTSB,Information verified through data from accident investigation authorities,ASTR,Sunday,5,5,USA,FLL,KFLL,HSP,KHSP,IAI 1125 Astra SPSkyJet EliteN1125A051,IAI 1125 Astra SP,SkyJet Elite,N1125A,051,1990,null,null,null,Israel,1984,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,IAI 1125 Astra SPSkyJet EliteN1125A051
318650,2023-05-19,1100,Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,null,null,"Information is only available from news, social media or unofficial sources",SC7,Friday,0,2,Uganda,null,null,null,null,Shorts SC.7 Skyvan 3A-100Uganda Peoples Defence ForceAF-519SH.1901,Shorts SC.7 Skyvan 3A-100,Uganda Peoples Defence Force,AF-519,SH.1901,1972,null,null,null,United Kingdom,1963,463f2afc-b0f1-4993-ae58-02d1ae5d1134,Shorts SC.7 Skyvan 3A-100Uganda Peoples Defence ForceAF-519SH.1901
318643,2023-07-08,1500,Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,0,Substantial,Accident,Vårgårda,Take off,Private,null,null,SHK,"Information is only available from news, social media or unofficial sources",AN2,Saturday,0,5,Sweden,null,null,null,null,Antonov An-2RAviatörsföreningen Antonov 2SE-KCE1G189-59,Antonov An-2R,Aviatörsföreningen Antonov 2,SE-KCE,1G189-59,1981,Shvetsov ASh-62IR,null,null,Ukraine,1947,7d784c5f-aee0-447f-878c-1b55125b60d7,Antonov An-2RAviatörsföreningen Antonov 2SE-KCE1G189-59
318648,2023-06-04,1500,Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,null,"Islip-Long Island MacArthur Airport, NY",NTSB,Information verified through data from accident investigation authorities,C560,Sunday,4,4,USA,null,null,ISP,KISP,Cessna 560 Citation VEncore Motors of Melbourne IncN611VG560-0091,Cessna 560 Citation V,Encore Motors of Melbourne Inc,N611VG,560-0091,1990,Pratt & Whitney Canada JT15D-5,null,null,USA,1987,aec827a3-4665-4889-83bf-099ae82c5cdf,Cessna 560 Citation VEncore Motors of Melbourne IncN611VG560-0091
389190,2024-05-31,null,Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,N'Djamena Airport,Abuja-Nnamdi Azikiwe International Airport,null,Information verified through data from accident investigation authorities,A332,Friday,0,null,Chad,NDJ,FTTJ,ABV,DNAA,Airbus A330-203Air FranceF-GZCL519,Airbus A330-203,Air France,F-GZCL,519,2003,GE CF6-80E1A3,null

Uma vez recuperado o aircraft_id, podemos dropar as colunas relacionadas ao dados de aeronaves, pois eles são salvos na tabela aircraft_type_dim e recuperáveis via aircraft_id

In [0]:
# Dropando colunas desnecessárias
accidents_gold_fact = accidents_gold_fact.drop('aircraft_type','owner_operator','registration','msn','year_of_manufacture','engine_model','total_airframe_hrs','cycles','departure_airport','destination_airport','aircraft_type_code','weekday','manufacturer_country_name','year_of_first_flight_of_model','full_aircraft_id')

accidents_gold_fact.display()

index,date,time,other_fatalities,aircraft_damage,category,approx_location,phase,nature,investigating_agency,confidence_rating,nb_fatalities,nb_occupants,country_or_region,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO,total_airframe_hours,aircraft_id
386881,2024-04-23,1345,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,CENIPA,"Information is only available from news, social media or unofficial sources",0,5,Brazil,XAP,SBCH,ERM,SSER,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66
354203,2024-03-10,1452,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,NTSB,Information verified through data from accident investigation authorities,5,5,USA,FLL,KFLL,HSP,KHSP,null,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c
318650,2023-05-19,1100,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,"Information is only available from news, social media or unofficial sources",0,2,Uganda,null,null,null,null,null,463f2afc-b0f1-4993-ae58-02d1ae5d1134
318643,2023-07-08,1500,0,Substantial,Accident,Vårgårda,Take off,Private,SHK,"Information is only available from news, social media or unofficial sources",0,5,Sweden,null,null,null,null,null,7d784c5f-aee0-447f-878c-1b55125b60d7
318648,2023-06-04,1500,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,NTSB,Information verified through data from accident investigation authorities,4,4,USA,null,null,ISP,KISP,null,aec827a3-4665-4889-83bf-099ae82c5cdf
389190,2024-05-31,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,null,Information verified through data from accident investigation authorities,0,null,Chad,NDJ,FTTJ,ABV,DNAA,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8
343377,2023-08-09,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,null,Canada,null,null,null,null,null,8271fa51-79cb-4393-9518-47ef4167f387
351839,2024-02-18,2009,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,Aerocivil,"Information is only available from news, social media or unofficial sources",0,null,Atlantic Ocean,JFK,KJFK,BOG,SKBO,null,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7
318674,2023-04-17,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,0,Sudan,null,null,null,null,null,69d9ca9d-302e-49fb-90ca-ef471289bce9
344872,2023-08-24,1542,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,BEA,"Information is only available from news, social media or unofficial sources",0,6,null,PTP,TFFR,SBH,TFFJ,null,f22e1efe-d9f3-4eb6-b912-3c734f806173


Vamos tratar agora a coluna Date:
- A partir da data AAAA-MM-DD criamos a coluna date_id no formato AAAAMMDD, que é FK para a dimensão de data
- Após, dropamos a coluna date

In [0]:
# Criando a coluna date_id
accidents_gold_fact = accidents_gold_fact.withColumn('date_id',F.date_format('date','yyyyMMdd'))

# Dropando a coluna date
accidents_gold_fact = accidents_gold_fact.drop('date')

accidents_gold_fact.display()

index,time,other_fatalities,aircraft_damage,category,approx_location,phase,nature,investigating_agency,confidence_rating,nb_fatalities,nb_occupants,country_or_region,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO,total_airframe_hours,aircraft_id,date_id
386881,1345,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,CENIPA,"Information is only available from news, social media or unofficial sources",0,5,Brazil,XAP,SBCH,ERM,SSER,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66,20240423
354203,1452,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,NTSB,Information verified through data from accident investigation authorities,5,5,USA,FLL,KFLL,HSP,KHSP,null,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,20240310
318650,1100,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,"Information is only available from news, social media or unofficial sources",0,2,Uganda,null,null,null,null,null,463f2afc-b0f1-4993-ae58-02d1ae5d1134,20230519
318643,1500,0,Substantial,Accident,Vårgårda,Take off,Private,SHK,"Information is only available from news, social media or unofficial sources",0,5,Sweden,null,null,null,null,null,7d784c5f-aee0-447f-878c-1b55125b60d7,20230708
318648,1500,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,NTSB,Information verified through data from accident investigation authorities,4,4,USA,null,null,ISP,KISP,null,aec827a3-4665-4889-83bf-099ae82c5cdf,20230604
389190,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,null,Information verified through data from accident investigation authorities,0,null,Chad,NDJ,FTTJ,ABV,DNAA,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8,20240531
343377,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,null,Canada,null,null,null,null,null,8271fa51-79cb-4393-9518-47ef4167f387,20230809
351839,2009,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,Aerocivil,"Information is only available from news, social media or unofficial sources",0,null,Atlantic Ocean,JFK,KJFK,BOG,SKBO,null,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,20240218
318674,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,0,Sudan,null,null,null,null,null,69d9ca9d-302e-49fb-90ca-ef471289bce9,20230417
344872,1542,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,BEA,"Information is only available from news, social media or unofficial sources",0,6,null,PTP,TFFR,SBH,TFFJ,null,f22e1efe-d9f3-4eb6-b912-3c734f806173,20230824


Tratando a coluna country_or_region:
- Vamos criar uma coluna de country_code a partir de country_or_region que será FK para a tabela de dimensão country

In [0]:
# Transformando a coluna country_or_region em country_code

countries_gold_dim = spark.read.format('delta').load('dbfs:/user/hive/warehouse/gold_database.db/gold_countries_dim')

countries_gold_dim.display()


country_code,country_name
T2,Tuvalu
YK,Syria
EI,Ireland
F-Oma,Martinique
UK,Uzbekistan
VP-L,British Virgin Islands
TL,Central African Republic
T7,San Marino
OM,Slovakia
TG,Guatemala


Com base no country_name, fazemos join com a tabela countries_gold_dim para obter o country_code associado
e dropamos as demais colunas, ficando apenas com country_code

In [0]:

# Fazendo join com country_name para obter os country_code

cond = [accidents_gold_fact.country_or_region == countries_gold_dim.country_name]
accidents_gold_fact = accidents_gold_fact.join(countries_gold_dim,cond,'left')

#accidents_gold_fact.display()

# Dropando as colunas desnecessárias de country_name e de country_or_region
accidents_gold_fact = accidents_gold_fact.drop('country_name','country_or_region')

accidents_gold_fact.display()


index,time,other_fatalities,aircraft_damage,category,approx_location,phase,nature,investigating_agency,confidence_rating,nb_fatalities,nb_occupants,departure_airport_IATA,departure_airport_ICAO,destination_airport_IATA,destination_airport_ICAO,total_airframe_hours,aircraft_id,date_id,country_code
386881,1345,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,CENIPA,"Information is only available from news, social media or unofficial sources",0,5,XAP,SBCH,ERM,SSER,null,9e8e8a58-60cc-469d-bf1f-4de64007fe66,20240423,PP
354203,1452,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,NTSB,Information verified through data from accident investigation authorities,5,5,FLL,KFLL,HSP,KHSP,null,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,20240310,N
318650,1100,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,"Information is only available from news, social media or unofficial sources",0,2,null,null,null,null,null,463f2afc-b0f1-4993-ae58-02d1ae5d1134,20230519,5X
318643,1500,0,Substantial,Accident,Vårgårda,Take off,Private,SHK,"Information is only available from news, social media or unofficial sources",0,5,null,null,null,null,null,7d784c5f-aee0-447f-878c-1b55125b60d7,20230708,SE
318648,1500,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,NTSB,Information verified through data from accident investigation authorities,4,4,null,null,ISP,KISP,null,aec827a3-4665-4889-83bf-099ae82c5cdf,20230604,N
389190,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,null,Information verified through data from accident investigation authorities,0,null,NDJ,FTTJ,ABV,DNAA,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8,20240531,TT
343377,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,null,null,null,null,null,null,8271fa51-79cb-4393-9518-47ef4167f387,20230809,C
351839,2009,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,Aerocivil,"Information is only available from news, social media or unofficial sources",0,null,JFK,KJFK,BOG,SKBO,null,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,20240218,atl
318674,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,"Information is only available from news, social media or unofficial sources",0,0,null,null,null,null,null,69d9ca9d-302e-49fb-90ca-ef471289bce9,20230417,ST
344872,1542,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,BEA,"Information is only available from news, social media or unofficial sources",0,6,PTP,TFFR,SBH,TFFJ,null,f22e1efe-d9f3-4eb6-b912-3c734f806173,20230824,null


De posse de todas as FK (as de tempo e aeroporto já eram FKs "nativas"), reordenamos as colunas e salvamos no schema definido

In [0]:
# Reorndenando as colunas e salvando no schema do fato
accidents_gold_fact = accidents_gold_fact.select('index','aircraft_id','other_fatalities','aircraft_damage','category','approx_location','phase','nature','departure_airport_ICAO','departure_airport_IATA','destination_airport_ICAO','destination_airport_IATA','nb_fatalities','nb_occupants','country_code','investigating_agency','confidence_rating','date_id','time')

data = accidents_gold_fact.rdd
accidents_gold_fact = spark.createDataFrame(data,schema)


accidents_gold_fact.display()


index,aircraft_id,other_fatalities,aircraft_damage,category,approx_location,phase,nature,departure_airport_ICAO,departure_airport_IATA,destination_airport_ICAO,destination_airport_IATA,nb_fatalities,nb_occupants,country_code,investigation_agency,confidence_rating,date_id,time_id
386881,9e8e8a58-60cc-469d-bf1f-4de64007fe66,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,SBCH,XAP,SSER,ERM,0,5,PP,CENIPA,"Information is only available from news, social media or unofficial sources",20240423,1345
354203,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,KFLL,FLL,KHSP,HSP,5,5,N,NTSB,Information verified through data from accident investigation authorities,20240310,1452
318650,463f2afc-b0f1-4993-ae58-02d1ae5d1134,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,null,null,null,0,2,5X,null,"Information is only available from news, social media or unofficial sources",20230519,1100
318643,7d784c5f-aee0-447f-878c-1b55125b60d7,0,Substantial,Accident,Vårgårda,Take off,Private,null,null,null,null,0,5,SE,SHK,"Information is only available from news, social media or unofficial sources",20230708,1500
318648,aec827a3-4665-4889-83bf-099ae82c5cdf,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,null,null,KISP,ISP,4,4,N,NTSB,Information verified through data from accident investigation authorities,20230604,1500
389190,4f2da891-6bce-4f2c-aa99-001e5d306fd8,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,FTTJ,NDJ,DNAA,ABV,0,null,TT,null,Information verified through data from accident investigation authorities,20240531,null
343377,8271fa51-79cb-4393-9518-47ef4167f387,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,null,null,null,0,null,C,null,"Information is only available from news, social media or unofficial sources",20230809,null
351839,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,KJFK,JFK,SKBO,BOG,0,null,atl,Aerocivil,"Information is only available from news, social media or unofficial sources",20240218,2009
318674,69d9ca9d-302e-49fb-90ca-ef471289bce9,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,null,null,null,0,0,ST,null,"Information is only available from news, social media or unofficial sources",20230417,null
344872,f22e1efe-d9f3-4eb6-b912-3c734f806173,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,TFFR,PTP,TFFJ,SBH,0,6,null,BEA,"Information is only available from news, social media or unofficial sources",20230824,1542


##Avaliando a qualidade dos dados
- Testar se index não possui nulo
- Testar se aircraft_id não possui nulo
- Testar se as 4 colunas de aeroportos estão com os códigos ICAO e IATA íntegros
- Testar se o número de outras vítimas é não negativo
- Testar se o número de vítimas é não negativo
- Testar se o número de ocupantes é não negativo
- Testar se o número de ocupantes é maior ou igual ao número de vítimas (não podemos ter mais vítimas que ocupantes)
- Testar se a maior data presente na FK é menor que a data atual
- Testar se o tempo presente na FK está entre 0000 e 2359
- Testar se as FKs estão dentre os valores possíveis comparando dataframe a dataframe das dimensões

In [0]:
# Testando a qualidade dos dados

test_col_not_null(accidents_gold_fact,'index')
test_col_not_null(accidents_gold_fact,'aircraft_id')
test_ICAO_codes(accidents_gold_fact,'destination_airport_ICAO')
test_ICAO_codes(accidents_gold_fact,'departure_airport_ICAO')
test_IATA_codes(accidents_gold_fact,'destination_airport_IATA')
test_IATA_codes(accidents_gold_fact,'departure_airport_IATA')
test_value_range(accidents_gold_fact,"other_fatalities",">= 0 or other_fatalities is null")
test_value_range(accidents_gold_fact,"nb_fatalities",">= 0 or nb_fatalities is null")
test_value_range(accidents_gold_fact,"nb_occupants",">= 0 or nb_occupants is null")
test_value_range(accidents_gold_fact,"nb_occupants",">= nb_fatalities or nb_occupants is null or nb_fatalities is null")
test_biggest_date_before_current_date(accidents_gold_fact,"date_id")
test_value_range(accidents_gold_fact,"time_id",">= 0000 or time_id is null")
test_value_range(accidents_gold_fact,"time_id","<= 2359 or time_id is null")


# Verificando se as FKs estão dentro de valores padrão das outras tabelas

aircraft_dim_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_aircraft_type_dim')
airport_dim_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_airport_dim')
countries_dim_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_countries_dim')
date_dim_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_data_dim')
time_dim_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/gold_database.db/gold_time_dim')


assert_dataframes(accidents_gold_fact,'aircraft_id',aircraft_dim_df,'aircraft_id')
assert_dataframes(accidents_gold_fact,'destination_airport_ICAO',airport_dim_df,'airport_ICAO_code')
assert_dataframes(accidents_gold_fact,'departure_airport_ICAO',airport_dim_df,'airport_ICAO_code')
assert_dataframes(accidents_gold_fact,'destination_airport_IATA',airport_dim_df,'airport_IATA_code')
assert_dataframes(accidents_gold_fact,'departure_airport_IATA',airport_dim_df,'airport_IATA_code')
assert_dataframes(accidents_gold_fact,'country_code',countries_dim_df,'country_code')
assert_dataframes(accidents_gold_fact,'date_id',date_dim_df,'date_id')
assert_dataframes(accidents_gold_fact,'time_id',time_dim_df,'time_id')




Avaliando a condição index não contem nulos
Avaliando a condição aircraft_id não contem nulos
Avaliando a condição destination_airport_ICAO contem 4 caracteres
Avaliando a condição departure_airport_ICAO contem 4 caracteres
Avaliando a condição destination_airport_IATA contem 3 caracteres
Avaliando a condição departure_airport_IATA contem 3 caracteres
Avaliando a condição other_fatalities >= 0 or other_fatalities is null
Avaliando a condição nb_fatalities >= 0 or nb_fatalities is null
Avaliando a condição nb_occupants >= 0 or nb_occupants is null
Avaliando a condição nb_occupants >= nb_fatalities or nb_occupants is null or nb_fatalities is null
Avaliando a condição date_id menor que a data atual
Avaliando a condição time_id >= 0000 or time_id is null
Avaliando a condição time_id <= 2359 or time_id is null
Comparando a coluna aircraft_id do DF1 com aircraft_id do DF
Comparando a coluna destination_airport_ICAO do DF1 com airport_ICAO_code do DF
Comparando a coluna departure_airport_ICAO

## Registrando dados no banco
Após aprovação, salvamos o dado sanitizado no banco e verificamos se tivemos sucesso

In [0]:
# Cria o banco de dados se ele não existir
database_name = 'gold_database'
table_name = 'gold_accidents_fact'


spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Salva o DataFrame como tabela Delta
accidents_gold_fact.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable(
    f"{database_name}.{table_name}"
)

In [0]:
%sql
SELECT * from gold_database.gold_accidents_fact

index,aircraft_type,other_fatalities,aircraft_damage,category,approx_location,phase,nature,departure_airport_ICAO,departure_airport_IATA,destination_airport_ICAO,destination_airport_IATA,nb_fatalities,nb_occupants,country_code,investigation_agency,confidence_rating,date_id,time_id,aircraft_id
386881,null,0,Substantial,Accident,"Erechim Airport (ERM), RS",Landing,Private,SBCH,XAP,SSER,ERM,0,5,PP,CENIPA,"Information is only available from news, social media or unofficial sources",20240423,1345,9e8e8a58-60cc-469d-bf1f-4de64007fe66
354203,null,0,Destroyed,Accident,"near Ingalls Field Airport (KHSP), Hot Springs, VA",Approach,Private,KFLL,FLL,KHSP,HSP,5,5,N,NTSB,Information verified through data from accident investigation authorities,20240310,1452,d5648ad0-dfa4-44ed-a84f-eaf2ab475c9c
318650,null,0,"Destroyed, written off",Accident,near Kalongo Airstrip,Landing,Military,null,null,null,null,0,2,5X,null,"Information is only available from news, social media or unofficial sources",20230519,1100,463f2afc-b0f1-4993-ae58-02d1ae5d1134
318643,null,0,Substantial,Accident,Vårgårda,Take off,Private,null,null,null,null,0,5,SE,SHK,"Information is only available from news, social media or unofficial sources",20230708,1500,7d784c5f-aee0-447f-878c-1b55125b60d7
318648,null,0,"Destroyed, written off",Accident,"near Montebello, VA",En route,Private,null,null,KISP,ISP,4,4,N,NTSB,Information verified through data from accident investigation authorities,20230604,1500,aec827a3-4665-4889-83bf-099ae82c5cdf
389190,null,0,Substantial,Accident,N'Djamena Airport (NDJ/FTTJ),Standing,Passenger - Scheduled,FTTJ,NDJ,DNAA,ABV,0,null,TT,null,Information verified through data from accident investigation authorities,20240531,null,4f2da891-6bce-4f2c-aa99-001e5d306fd8
343377,null,0,Unknown,Accident,"Comox Airport, British Columbia. (YQQ/CYQQ)",Standing,Military,null,null,null,null,0,null,C,null,"Information is only available from news, social media or unofficial sources",20230809,null,8271fa51-79cb-4393-9518-47ef4167f387
351839,null,0,Unknown,Accident,E of Florida,En route,Passenger - Scheduled,KJFK,JFK,SKBO,BOG,0,null,atl,Aerocivil,"Information is only available from news, social media or unofficial sources",20240218,2009,04e2160e-034b-4b97-a4e5-e6cd6bb3a1e7
318674,null,0,"Destroyed, written off",OT,Khartoum International Airport (KRT),Standing,Military,null,null,null,null,0,0,ST,null,"Information is only available from news, social media or unofficial sources",20230417,null,69d9ca9d-302e-49fb-90ca-ef471289bce9
344872,null,0,Substantial,Accident,Saint-Barthélémy-Rémy de Haenen Airport (SBH/TFFJ),Landing,Passenger - Scheduled,TFFR,PTP,TFFJ,SBH,0,6,null,BEA,"Information is only available from news, social media or unofficial sources",20230824,1542,f22e1efe-d9f3-4eb6-b912-3c734f806173


In [0]:
'''
Finalizando build_accidents_fact
'''

'\nFinalizando build_accidents_fact\n'